In [7]:
import sys
sys.path.append(os.environ["BUILD_WORKSPACE_DIRECTORY"])
import json
import os
import matplotlib.pyplot as plt
from PIL import Image
root_dir = '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/'
data_dirs = [x[0] for x in os.walk(root_dir)][1:]
print(data_dirs)
from core.infra.cloud.gcs_cv2_utils import upload_cv2_image_to_gcs
from core.infra.cloud.gcs_utils import dump_to_gcs, get_storage_client
output_path = "gs://voxel-users/reza/ergo_syntheticdata_downsample"
storage_client = get_storage_client()

['/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0002', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0005', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0004', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_uscold-laredo-dock01', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0008', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0007', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0006', '/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_down

In [8]:
import os

from core.perception.pose.api import FastPose, PoseModel

class PoseModelSyn(PoseModel):
    def __call__(self, frame, annotation):
        box = [annotation['bbox'][0],
        annotation['bbox'][1],
        annotation['bbox'][0] + annotation['bbox'][2], 
        annotation['bbox'][1] + annotation['bbox'][3]]
        boxes = []
        boxes.append(box)
        boxes = torch.from_numpy(np.asarray(boxes))
        scores = []
        scores.append(1)
        scores = torch.from_numpy(np.asarray(scores))
        results = (
            self._process(frame, boxes, scores) if boxes is not None else []
        )
        pose = None
        for result in results:
            pose = self._get_pose_from_result(result)
        
        return pose



In [9]:
keypoints = [
            "nose",
            "left_eye",
            "right_eye",
            "left_ear",
            "right_ear",
            "left_shoulder",
            "right_shoulder",
            "left_elbow",
            "right_elbow",
            "left_wrist",
            "right_wrist",
            "left_hip",
            "right_hip",
            "left_knee",
            "right_knee",
            "left_ankle",
            "right_ankle"
            ]

In [ ]:
import numpy as np
import torch
import cv2
pose_model = PoseModelSyn('/home/reza_voxelsafety_com/experiments/ergonomic/model/fast_res50_256x192.pth')
for i in range(len(data_dirs)):
    print(data_dirs[i])
    data = json.load(open(f"{data_dirs[i]}/annotations.json"))
    save_subfolder = data_dirs[i].split('/')[-1]
    img_dict = {}
    for name in data['images']:
        img_dict[name['id']] = name['file_name']
    annotations = data['annotations']
    annotations_new = []
    for anno in annotations:
        keypoints_model = []
        img_name = img_dict[anno['image_id']]
        sample_image = Image.open(os.path.join(data_dirs[i], img_name))
        
        if anno['category_id'] == 0:
#             print(f"keypoints: {anno['keypoints']}")
            box = anno['bbox']
            img_array = np.asarray(sample_image)[...,:3]
            #temp = cv2.cvtColor(np.ascontiguousarray(temp), cv2.COLOR_RGB2BGR)
            pose = pose_model(img_array, anno)
            if pose is not None:
            #img_draw = pose.draw(np.ascontiguousarray(img_array))
#             plt.imshow(img_draw)
#             plt.show()
                model_output = pose.to_dict()
                for keyp in keypoints:
                    keypoints_model.append(model_output[keyp]['x'])
                    keypoints_model.append(model_output[keyp]['y'])
                    keypoints_model.append(model_output[keyp]['confidence'])
                actor_img = sample_image.crop((box[0], box[1], box[0] + box[2], box[1] + box[3]))
                anno['keypoints'] = keypoints_model
    #             print(f"keypoints of alphapose: {keypoints_model}")
                annotations_new.append(anno)
            else:
                anno['keypoints'] = []
                annotations_new.append(anno)
    data['annotations'] = annotations_new
    gcs_label_path = f"{output_path}/{save_subfolder}/annotations_model.json"
    dump_to_gcs(
            gcs_label_path,
            json.dumps(data),
            content_type="application/json",
    )

/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0002
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0005
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0004
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_uscold-laredo-dock01
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0008
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-pooler-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0007
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah-bloomingdale-0006
/home/reza_voxelsafety_com/experiments/ergonomic/ergo_syntheticdata_downsample/_americold-savannah